In [1]:
# Lightning related imports

# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning fsspec --no-deps --target=$nb_path
# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning
!pip install torchtext==0.8.0 torch==1.7.0 torchvision==0.8 pytorch-lightning==1.2.2
!pip install neptune-client==0.5.1

# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning
# https://github.com/PyTorchLightning/pytorch-lightning/issues/6210

import pytorch_lightning
print(pytorch_lightning.__version__)
from pytorch_lightning import Trainer
from pytorch_lightning.loggers.neptune import NeptuneLogger
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

     |████████████████████████████████| 7.0MB 14.6MB/s 
     |████████████████████████████████| 776.8MB 15kB/s 
     |████████████████████████████████| 11.8MB 212kB/s 
     |████████████████████████████████| 819kB 55.7MB/s 
     |████████████████████████████████| 276kB 58.6MB/s 
     |████████████████████████████████| 112kB 56.6MB/s 
     |████████████████████████████████| 1.3MB 42.8MB/s 
     |████████████████████████████████| 296kB 56.6MB/s 
     |████████████████████████████████| 143kB 60.2MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=984fd7bb27a07446c96351c3eb205efcefeb58c8affe7b33b246a73bc341f589
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
ERROR: pytorch-lightning 1.2.2 has requirement future>=0.17.1, but you'll have future 0.16.0 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
 

In [2]:
# !pip install torch==1.7.0 torchvision==0.8 torchtext==0.8
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-geometric
!pip install ogb

     |████████████████████████████████| 2.8MB 24.1MB/s 
     |████████████████████████████████| 1.6MB 32.6MB/s 
     |████████████████████████████████| 194kB 17.4MB/s 
     |████████████████████████████████| 235kB 37.3MB/s 
     |████████████████████████████████| 2.2MB 51.0MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
  Created wheel for outdated: filename=outdated-0.2.0-cp37-none-any.whl size=4962 sha256=f5ea4dc14373409d1e51fb945df470d8f5a7ff44b4794a81db61e8d13dd3fe84
  Stored in directory: /root/.cache/pip/wheels/fd/7c/ef/814f514d31197310872b5abf353feb8fef9d67ee658e1e7e39
  Created wheel for littleutils: filename=littleutils-0.2.2-cp37-none-any.whl size=7051 sha256=6470fd0332f9c94aa4361639bc29db78402b50dad578a6ff41f2ccb8cdf6b00c
  Stored in directory: /root/.cache/pip/wheels/53/16/9f/ac67d15c40243754fd73f620e1b9b6dedc20492ecc19a2bae1
Successfully built outdated littleutils


In [4]:
%load_ext autoreload
%autoreload 2
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
# from scikitplot.metrics import plot_confusion_matrix

from lightning_model import LightningPAN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Testing if the lightning_model script gets refreshed correctly
LightningPAN.training_epoch_end("hello",'Hle')


HEASDFSD?


TypeError: ignored

In [ ]:

# loading params
with open('parameters.json') as json_file:
    parameters = json.load(json_file)
print(parameters)
datasetname = parameters["dataset_name"]
phi = parameters["phi"]
runs = parameters["runs"]
batch_size = parameters["batch_size"]
filter_size = parameters["maximum_path_size"]+1
learning_rate = parameters["learning_rate"]
weight_decay = parameters["weight_decay"]
pool_ratio = parameters["pool_ratio"]
nhid = parameters["nhid"]
epochs = parameters["epochs"]
LearningRateMonitor_Params = {'logging_interval': 'epoch'}

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: {}'.format(device))

model = LightningPAN(9, 1, nhid=nhid,ratio=pool_ratio,filter_size=filter_size)

lr_logger = LearningRateMonitor(**LearningRateMonitor_Params)
neptune_logger = NeptuneLogger(
                api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMDk5YjVmYzYtNTU0My00MzhkLWJiYTAtMGM4ZGVhZmEyMTZiIn0=",
                project_name='hvergnes/PAN',
                close_after_fit=False,
                params=parameters, # your hyperparameters, immutable
                tags=['PAN', 'fixed epoch end issues'],  # tags
                upload_source_files=["parameters.json", "lightning_model.py"]
                )

trainer = Trainer(
    max_epochs=epochs,
    logger=neptune_logger,
    callbacks=[lr_logger],
    gpus=1,
    # fast_dev_run=True,
)

model.cuda()
trainer.fit(model)
trainer.test(model)

NeptuneLogger will work in online mode
GPU available: True, used: True
TPU available: None, using: 0 TPU cores


{'dataset_name': 'ogbg-molhiv', 'phi': 0.3, 'runs': 1, 'num_workers': 0, 'batch_size': 32, 'maximum_path_size': 4, 'learning_rate': 0.001, 'weight_decay': 0.001, 'pool_ratio': 0.5, 'nhid': 128, 'epochs': 100}
Device: cuda
https://ui.neptune.ai/hvergnes/PAN/e/PAN-127



   | Name         | Type              | Params
----------------------------------------------------
0  | criterion    | BCEWithLogitsLoss | 0     
1  | atom_encoder | AtomEncoder       | 5.5 K 
2  | conv1        | PANConv           | 4.2 K 
3  | norm1        | Norm              | 384   
4  | pool1        | PANPooling        | 135   
5  | drop1        | PANDropout        | 0     
6  | conv2        | PANConv           | 16.5 K
7  | norm2        | Norm              | 384   
8  | pool2        | PANPooling        | 133   
9  | drop2        | PANDropout        | 0     
10 | conv3        | PANConv           | 16.5 K
11 | norm3        | Norm              | 384   
12 | pool3        | PANPooling        | 133   
13 | lin1         | Linear            | 8.3 K 
14 | bn1          | BatchNorm1d       | 128   
15 | lin2         | Linear            | 2.1 K 
16 | bn2          | BatchNorm1d       | 64    
17 | lin3         | Linear            | 33    
----------------------------------------------------


Downloaded 0.00 GB: 100%|██████████| 3/3 [00:01<00:00,  2.39it/s]


Extracting dataset/hiv.zip
Processing...
Loading necessary files...
This might take a while.


...

Converting graphs into PyG objects...


100%|██████████| 41127/41127 [00:00<00:00, 61452.94it/s]


Saving...
Done!


/usr/local/lib/python3.7/dist-packages/torch_sparse/storage.py:382: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  ptr = mask.nonzero().flatten()


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: RuntimeWarning: You are using `LearningRateMonitor` callback with models that have no learning rate schedulers. Please see documentation for `configure_optimizers` method.
  warnings.warn(*args, **kwargs)


In [ ]:
import os
print(os.listdir())
torch.save(model.state_dict(), 'model.pt')
print(os.listdir())